<a href="https://colab.research.google.com/github/AldoKwamibar/Pd-membranes-permeability/blob/main/Palladium_Membrane_ML_Preprocessing_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# """
# This section is a material informatics project aiming to build a quantitative composition-structure-property (QSPR) relationship between palladium-based alloys membranes and their hydrogen permeability.
# ...
# """

In [ ]:
%%time
# Utilities to visualize periodic table and Element Counts in a material Informatics database
!pip install pymatgen matminer

In [ ]:
%%time
# Import useful packages and dependencies
import os
import pandas as pd
import numpy as np
import scipy as sc
import re
import unicodedata
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format='retina' #Allow image to have high resolution
np.random.seed(42) # Ensure reproducibility

In [ ]:
%%time
from pymatgen.core import Element, Composition
from matminer.featurizers.composition import ElementFraction
from matminer.featurizers.conversions import StrToComposition

In [ ]:
%%time
# Mounting google drive to import and export file from and to it
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# def  create_folder(path):
#   if not os.path.exists(path):
#     os.makedirs(path)
#     print(f'Directory {path} created.')
#   else:
#     print(f'Directory {path} already exists.')
# create_folder('/content/drive/Dataset_MaterialInformatics_PhD')
# create_folder('/content/drive/result_MaterialInformatics_PhD')

# os.chdir('/content/drive/Colab Notebooks/')
# print('Work space:', os.getcwd())

In [ ]:
%%time
# Setting this options allow Pandas to display every columns. By default only few are displayed.
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
%%time
data_path = '/content/drive/MyDrive/PhD_Pd alloy prediction/Palladium_Alloy_Membranes dataset_QSPR_ML.csv'
raw_data_1 = pd.read_csv(data_path)
print(f'Original DataFrame shape: {raw_data_1.shape}')

In [ ]:
%%time
raw_data_1.info()

In [ ]:
# Displaying columns-name-corrected dataset
%%time
raw_data_1.head()

In [ ]:
%%time
raw_data_1.describe(include=[np.number])

In [ ]:
# Renaming some column names in the dataset to remove newline '\n' sign
%%time
rename_dict = {'Composition\n type': 'Composition type',
              'Thickness \n(micron)':'Thickness (micron)',
              'Latice parameter (nm)': 'Lattice parameter (nm)',
              'Diffusion \ncoefficient\n(cm^2.s^-1)': 'Diffusion coefficient(cm^2.s^-1)',
              raw_data_1.columns.values[13]: 'Pressure difference P1^n-P2^n (Pa^n)',
              'Inverse temperature \n1E3/T (1E3 K-1)': 'Inverse temperature (1000/T in (1000 K^-1))',
              raw_data_1.columns.values[14]: 'Pressure exponent (n)',
              'Hydrogen Permeation Flux\n(mol.m^-2.s^-1)': 'Hydrogen Permeation Flux (mol.m^-2.s^-1)',
              'Permeation \nActivation Energy (KJ/mol)': 'Permeation Activation Energy (KJ/mol)',
              'Bravais lattice in as-manufactured state':'Bravais lattice'}
raw_data_1=raw_data_1.rename(columns=rename_dict)

In [ ]:
# Dropping some columns
%%time
df = raw_data_1.copy()
columns_to_drop = [#Not sufficient data were recovered from the literature to populate this columns (diffusivity, solubility, Activation Energy, etc.)
                  #They would have been very helpfull.
    'fcc lattice parameter for Cu alloys', 'Bravais lattice in as-annealed form for PdCu based membranes', 'Inverse temperature (1000/T in (1000 K^-1))', 'Diffusion coefficient(cm^2.s^-1)', 'Effective area (cm^2)', 'Membrane support', 'Type of support', 'Gases', 'Solubility constant Ks (Pa^-0.5)', 'Hydrogen Permeation Flux (mol.m^-2.s^-1)', 'Permeation Activation Energy (KJ/mol)', 'Hydrogen Permeance (mol.m^-2.s^-1.Pa^-0.5)', 'Feed Flow rate (mL.min^-1)', 'Hydrogen transfer rate s.c.f.h.(mil).(sq.ft.)^-1', 'Conversion factor', 'Number','Unnamed: 31']
df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])
raw_data_1_col_dropped = df

In [ ]:
# Ckecking NaN in the 'Alloy Formula' column. If there are unfilled rows in that column, so they are considered null and can be removed.
%%time
df2 = raw_data_1_col_dropped.copy()
bool_nans_formula=df2['Alloy Formula'].isnull()
df2 = df2.drop(df2.loc[bool_nans_formula].index, axis=0).reset_index(drop=True)
raw_data_1_col_nan_dropped=df2
# Drop the rows of the DataFrame which contain NaNs
print(f'DataFrame shape before dropping NaNs: {raw_data_1_col_dropped.shape}')
print(f'DataFrame shape after NaNs are dropped: {df2.shape}')
print(f'raw_data_1_col_nan_dropped shape after dropping NaNs: {raw_data_1_col_nan_dropped.shape}')

In [ ]:
# Changing data types: Here, for example the thickness that should be a numeric is in object type
%%time
raw_data_1_col_nan_dropped.dtypes

In [ ]:
# Converting data types to the proper category. Numerical datatypes are converted to numerics np.float64 and string types remains 'str'
%%time
type_dict={'Alloy Formula':str,
           'Composition type':str,
           'Form':str,
           'Composition in mole percent': str,
           'Thickness (micron)': np.float64,
           'Lattice parameter (nm)':np.float64,
           'Bravais lattice':str,
           'Temperature (°C)':np.float64,
           'Fabrication technique':str,
           'Pressure difference P1^n-P2^n (Pa^n)':np.float64,
           'Pressure exponent (n)':np.float64,
           'Hydrogen permeability (mol.m^-1.s^-1.Pa^-n)':np.float64}

raw_data_1_col_nan_dropped = raw_data_1_col_nan_dropped.astype(type_dict)

In [ ]:
# Checking if everything went correctly
%%time
raw_data_1_col_nan_dropped.dtypes

In [ ]:
# The columns ['Composition type', 'Form', 'Bravais lattice in as-manufactured state', 'Fabrication technique'] contains some inconsitencies
#  Homogenizing the Composition type column
%%time
print(raw_data_1_col_nan_dropped['Composition type'].unique())

ctinstance_dict={'mole fraction':'mole percent',
                 'mole fraction ':'mole percent',
                 'weight fraction':'weight percent',
                 'mole fractiion': 'mole percent',
                 'weight fraction ': 'weight percent',
                 'weight':'weight percent'}

raw_data_1_col_nan_dropped['Composition type'] = raw_data_1_col_nan_dropped['Composition type'].replace(ctinstance_dict)
print(raw_data_1_col_nan_dropped['Composition type'].unique())
# The nan represent empty lines

In [ ]:
# Homogenizing the Form column
%%time
print(raw_data_1_col_nan_dropped['Form'].unique())

raw_data_1_col_nan_dropped['Form'] = raw_data_1_col_nan_dropped['Form'].replace({'foils':'foil',
                                                                                 'Self-standing':'self-standing foil',
                                                                                 'Condensed foils':'condensed foil',
                                                                                 'free-hanging':'free-hanging foil',
                                                                                 'foils//ecthed foils':'untouched foil and etched foil',
                                                                                 'foils on silicon wafer': 'foil prepared on Silicon wafer',
                                                                                 'foils/self-standing': 'self-standing foil'})

# Replace the nan values with 'foil'
raw_data_1_col_nan_dropped.loc[raw_data_1_col_nan_dropped['Form'].isnull(), 'Form'] = 'foil'
# Reprint the unique values
print(raw_data_1_col_nan_dropped['Form'].unique())

In [ ]:
# Homogenizing the Fabrication technique column
%%time

raw_data_1_col_nan_dropped['Fabrication technique'] = raw_data_1_col_nan_dropped['Fabrication technique'].str.capitalize()

raw_data_1_col_nan_dropped['Fabrication technique'] = raw_data_1_col_nan_dropped['Fabrication technique'].replace({
    'Arc melting in inert atmoosphere/cold rolling/intermediate vacuum annealing':'Arc melting in inert atmosphere/Cold rolling/intermediate vacuum annealing',
    'Commercial alfa aesar': 'Commercial Alfa Aesar',
    'Commercial/goodfellow co.':'Commercial Goodfellow Co.',
    'Melting casrting an cold rolling':'Melt casting and cold rolling',
    'Pellet from commercial pd powder/cold rolling':'Pellet from commercial Pd powder/Cold rolling',
    'Arc melting/cold rolling/cfubmsip (close field unballanced magnetron sputtering ion plating)':'Arc melting/cold rolling/CFUBMSIP (Close Field Unballanced Magnetron Sputtering Ion Plating)',
    'Dc magnetron sputtering':'DC-magnetron sputtering'})

print(raw_data_1_col_nan_dropped['Fabrication technique'].unique())

In [ ]:
# Homogenizing the Bravais lattice in as-manufactured state columns
%%time
raw_data_1_col_nan_dropped['Bravais lattice']=raw_data_1_col_nan_dropped['Bravais lattice'].replace({'bcc + fcc': 'bcc/fcc', 'fcc+bcc': 'bcc/fcc', 'fcc+bcc/bcc': 'bcc/fcc', 'bcc+fcc':'bcc/fcc'})
print(raw_data_1_col_nan_dropped['Bravais lattice'].unique())
# The nan values belongs to PdCu alloys, and we will be calculated those later on automatically

In [ ]:
%%time
# We are going to parse a little bit the formula to make them clean and ready for conversion to composition object using pymatgen or matminer packages
def clean_formula(formula):
    if isinstance(formula, str):
        # Normalize all unicode whitespace
        formula = ''.join(c for c in formula if not unicodedata.category(c).startswith('Z'))
        formula = re.sub(r"[^A-Za-z0-9.]", "", formula)
        return formula
    return formula

In [ ]:
%%time
# Applying the formula
raw_data_1_col_nan_dropped[['Composition in mole percent', 'Alloy Formula']] = raw_data_1_col_nan_dropped[['Composition in mole percent', 'Alloy Formula']].apply(clean_formula)

In [ ]:
%%time
raw_data_1_col_nan_dropped['Composition in mole percent'].unique()

In [ ]:
# Convert The Formula from weight percent to mole percent
# Define atomic weights globally for reuse
# Reference for periodic table: https://periodic.lanl.gov/index.shtml
# Reference on regular expressions with re module: https://docs.python.org/3/library/re.html
# Reference for formula to convert weight percent to atomic percent: https://plasmaterials.com/converting-atomic-percent-to-weight-percent-and-vice-versa/
%%time
ATOMIC_WEIGHTS = {
    'B': 10.81, 'Sc': 44.96, 'Ti': 47.88, 'V': 50.94, 'Cr': 52.00, 'Mn': 54.94, 'Fe': 55.85,
    'Co': 58.93, 'Ni': 58.69, 'Cu': 63.55, 'Zn': 65.39, 'Ga': 69.72, 'Y': 88.91, 'Zr': 91.22,
    'Nb': 92.91, 'Mo': 95.96, 'Tc': 98, 'Ru': 101.1, 'Rh': 102.9, 'Pd': 106.4, 'Ag': 107.9,
    'Cd': 112.4, 'In': 114.8, 'Sn': 118.7, 'Pb': 207.2, 'La': 138.9, 'Ce': 140.1, 'Pr': 140.9,
    'Nd': 144.2, 'Pm': 145, 'Sm': 150.4, 'Eu': 152.0, 'Gd': 157.2, 'Tb': 158.9, 'Dy': 162.5,
    'Ho': 164.9, 'Er': 167.3, 'Tm': 168.9, 'Yb': 173.0, 'Lu': 175.0, 'Hf': 178.5, 'Ta': 180.9,
    'W': 183.9, 'Re': 186.2, 'Os': 190.2, 'Ir': 192.2, 'Pt': 195.1, 'Au': 197.0, 'Al':26.98
}

def weight_percent_to_atomic_percent(formula: str) -> str: #Only formula in the form AxBy... are handled (Can be binary, ternary or more). Make sure x,y are weight  percentage
    """
    Convert a chemical formula from weight percentage (e.g., 'Pd60.00Cu40.00')
    to atomic percent (e.g., 'Pd47.25Cu52.75').

    Parameters:
        formula (str): Chemical formula with weight percentages.

    Returns:
        str: Reformatted chemical formula in atomic percent.
    """
    # Extract elements and their weight percentages
    tokens = re.findall(r'[A-Z][a-z]?|\d+\.?\d*', formula) # This returns a list of the element symbols and weight percentage

    # Build a dictionary with the key equal the element symbol and the values equal to the weight percentage and a atomic weight: {element:(weight percent, atomic weight) ...}
    element_data = {}
    for i in range(0, len(tokens) - 1, 2):
        element = tokens[i]
        weight_percent = float(tokens[i + 1])
        atomic_weight = ATOMIC_WEIGHTS.get(element) #get the values i.e atomic weight for the elements

        if atomic_weight is None:
            raise ValueError(f"Atomic weight for element '{element}' is not defined.")

        element_data[element] = (weight_percent, atomic_weight)

    # Convert weight % to atomic %
    denominator = sum(wt / at_wt for wt, at_wt in element_data.values())
    atomic_percent = {
        element: round((wt / at_wt) / denominator * 100, 2)
        for element, (wt, at_wt) in element_data.items()
    }

    # Create output string in the form: Pd47.25Cu52.75
    output = ''.join(f"{el}{val}" for el, val in atomic_percent.items())
    return output

In [ ]:
%%time
# Copy original dataframe
df3 = raw_data_1_col_nan_dropped.copy()

# Step 1: Separate before modifying
df3_weight = df3[df3['Composition type'] == 'weight percent'].copy()
df3_atomic = df3[df3['Composition type'] == 'mole percent'].copy()

# Step 2: Convert weight percent --> atomic percent
df3_weight['Composition in mole percent'] = df3_weight['Alloy Formula'].apply(weight_percent_to_atomic_percent)
df3_weight['Composition type'] = 'weight percent'

# Step 3: Use existing mole percent directly
df3_atomic['Composition in mole percent'] = df3_atomic['Alloy Formula']

# Step 4: Concatenate the two dataframes
df3_cleaned = pd.concat([df3_atomic, df3_weight], axis=0, ignore_index=True)

# sort values by ascending order (default) and reset index. This means we will have PdAg, PdAu and PdB, etc. before PdCe, PdCu etc.
df4 = df3_cleaned.sort_values(by='Composition in mole percent').reset_index(drop=True)
raw_data_1_col_nan_dropped=df4

In [ ]:
# We reparse the formulas for security
%%time
raw_data_1_col_nan_dropped['Composition in mole percent'] = raw_data_1_col_nan_dropped['Composition in mole percent'].apply(clean_formula)
raw_data_1_col_nan_dropped['Composition in mole percent'].unique()

In [ ]:
# Removing invalid n values i.e n>1 because they are not a frequent trend from the literature
%%time
df5 = raw_data_1_col_nan_dropped.copy()
bool_invalid_n = df5['Pressure exponent (n)']>0.5
print(f'The number of datapoints with n>0.5 is {raw_data_1_col_nan_dropped[bool_invalid_n].shape[0]}')

df5 = df5.drop(df5.loc[bool_invalid_n].index, axis = 0).reset_index(drop=True)
raw_data_1_col_nan_dropped = df5

print(f'The shape of the dataset with n<=0.5 is {raw_data_1_col_nan_dropped.shape[0]}')

In [ ]:
# Remove Study by Li because it was done using plasma heating, and El Shafie
%%time
df6=raw_data_1_col_nan_dropped.copy()
bool_invalid_ref = df6['Reference'].isin(['(Li et al., 2023)', '(El Shafie et al., 2020) '])
print(f'Total number of points in refs (Li et al., 2023) and (El Shafie et al., 2020) are: {raw_data_1_col_nan_dropped[bool_invalid_ref].shape[0]}')
print(f'The shape of the dataset before refs (Li et al., 2023) and (El Shafie et al., 2020) were removed is: {raw_data_1_col_nan_dropped.shape[0]}')
df6=df6.drop(df6.loc[bool_invalid_ref].index, axis = 0).reset_index(drop=True)
raw_data_1_col_nan_dropped=df6
print(f'The shape of the dataset after the removal of refs (Li et al., 2023) and (El Shafie et al., 2020): {raw_data_1_col_nan_dropped.shape[0]}')

In [ ]:
# Remove Study with hydrogen permeability >=1.0e-7
# Those data belongs to (Ievlev et al. 2012) and are about PdCu, but it is rare that the permeability of PdCu40 reach that level. This is due to their surface cleaning method.
# Keeping these values will distort the distribution of data
bool_perm_higher_than_e_power_minus_7 = raw_data_1_col_nan_dropped['Hydrogen permeability (mol.m^-1.s^-1.Pa^-n)']>=1.0e-7
print(f'The number of data points with permeability value superior >=1.0e-7 are removed is: {raw_data_1_col_nan_dropped[bool_perm_higher_than_e_power_minus_7].shape[0]}')
print(f'The size of the starting dataset is {raw_data_1_col_nan_dropped.shape[0]}')
df6 = df6.drop(df6.loc[bool_perm_higher_than_e_power_minus_7].index, axis = 0).reset_index(drop=True)
raw_data_1_col_nan_dropped=df6

# Verify if concerned data points have been removed successfully still
print(f'The shape of the dataset after data points with permeability>=1.0e-7 are removed is: {raw_data_1_col_nan_dropped.shape[0]}')

In [ ]:
%%time
raw_data_1_col_nan_dropped.isnull().sum()

In [ ]:
%%time
# Making sure all the missing lattice parameter values in the the 'Lattice parameter (nm)' belongs to the PdCu system :) Yes
raw_data_1_col_nan_dropped[raw_data_1_col_nan_dropped['Lattice parameter (nm)'].isnull()]['Alloy Formula'].unique()

In [ ]:
%%time
# Step 1: Parse string formulas into Composition objects
# This ensures robust parsing, even if formulas are inconsistently formatted
stc = StrToComposition(target_col_id="Composition_matrix")
print('Shape before formulas parsing into composition object:', raw_data_1_col_nan_dropped.shape)
raw_data_1_col_nan_dropped = stc.featurize_dataframe(raw_data_1_col_nan_dropped, col_id="Composition in mole percent")

# Step 2: Drop rows where parsing failed (Composition_matrix is NaN)
raw_data_1_col_nan_dropped = raw_data_1_col_nan_dropped.dropna(subset=["Composition_matrix"]).reset_index(drop=True)
print('Shape after formulas parsing into composition object:', raw_data_1_col_nan_dropped.shape)

# Step 3: Transform the composition object to element fraction matrix using Matminer's ElementFraction
ef = ElementFraction()
raw_data_1_col_nan_dropped = ef.featurize_dataframe(
    raw_data_1_col_nan_dropped,
    col_id="Composition_matrix",
    ignore_errors=True
)

# Step 4: Remove element columns with only zero values
raw_data_1_col_nan_dropped_matm = raw_data_1_col_nan_dropped.loc[:, (raw_data_1_col_nan_dropped != 0).any(axis=0)]
print('The shape of the dataframe with 0 element column dropped is:', raw_data_1_col_nan_dropped_matm.shape)

In [ ]:
%%time
#Print the datatype of the of the composition object. Is it a dictionary object?
print(type(raw_data_1_col_nan_dropped_matm.loc[0, 'Composition_matrix'].as_dict()))

In [ ]:
%%time
# In other to filling the NaN in the lattice paramter column, we will be applying Matminer to handle this much easily
# Estimating the lattice parameter of missing alloys

# Write a function to estimate the lattice parameter of PdCu alloy systems based on their atomic composition
# The lattice parameter of FCC PdCu obeys Vegard's law of a = (2.75*1.0e-3)*X_Pd + 3.62 (in Angström) where X_Pd is the mole percentage
# The lattice paramter of BCC PdCu obeys Vegard's law of a = (1.57*1.0e-3)*X_Pd + 2.90 (in Angström) where X_Pd is the mole percentage
# Reference in (Al-Mufachi et al., 2015), doi: https://doi.org/10.1016/j.memsci.2015.07.015

# Determine the boundaries of the phase diagram of PdCu
# Reference in: (Subramanian and Laughlin, 1991), https://www.andrew.cmu.edu/user/dl0p/laughlin/pdf/128.pdf
# fcc (%Pd): 0-23.411513859275052
# B2+fcc (%Pd): 23.411513859275052-31.194029850746263
# B2 (%Pd): 31.194029850746263-49.53091684434967
# B2+fcc (%Pd): 49.53091684434967-65.09594882729209
# fcc(%Pd): 65.09594882729209

def PdCu_lattice(raw_data_1_col_nan_dropped_matm):
    """
    Estimates the lattice parameter for PdCu alloys with missing values.
    Uses phase diagram boundaries to determine structure and lattice parameter.
    Assuming that before conducting a measurment with mixed bcc/fcc alloy, researcher would anneal completely to B2 intermetallic
    """
    # Identify rows with missing lattice parameters
    dataframe_block = raw_data_1_col_nan_dropped_matm.loc[raw_data_1_col_nan_dropped_matm['Lattice parameter (nm)'].isnull()].copy()

    # Define Pd-Cu phase diagram ranges
    Pd_Cu = {'Pd', 'Cu'}
    fcc_1_range = [0, 23.4115]
    fcc_2_range = [65.0959, 100]
    B2_fcc_range_1 = [23.4115, 31.1940]
    B2_fcc_range_2 = [49.5309, 65.0959]
    B2_range = [31.1940, 49.5309]

    # Define lattice parameter equations
    def estimate_lattice(row):
        # Ensure composition is PdCu only
        element_dict = row['Composition_matrix'].as_dict()
        if set(element_dict.keys()) == Pd_Cu:
            amount_Pd = element_dict.get('Pd', 0)

            # Determine phase and calculate correponding lattice parameter
            if min(fcc_1_range) <= amount_Pd <= max(fcc_1_range) or min(fcc_2_range) <= amount_Pd <= max(fcc_2_range):
                return 'fcc', ((2.75e-3) * amount_Pd + 3.62) / 10 # We want the answer in nm instead of Angstrom
            elif min(B2_range) <= amount_Pd <= max(B2_range):
                return 'bcc', ((1.57e-3) * amount_Pd + 2.90) / 10
            elif min(B2_fcc_range_1) <= amount_Pd <= max(B2_fcc_range_1) or min(B2_fcc_range_2) <= amount_Pd <= max(B2_fcc_range_2):
                return 'bcc/fcc', (0.5 * ((2.75e-3) * amount_Pd + 3.62) + 0.5 * ((1.57e-3) * amount_Pd + 2.90)) / 10
            else:
                return 'Unknown', np.nan
        return row['Bravais lattice'], row['Lattice parameter (nm)']

    # Apply the function to determine phase & lattice parameter
    dataframe_block[['Bravais lattice', 'Lattice parameter (nm)']] = dataframe_block.apply(
        estimate_lattice, axis=1, result_type="expand"
    )

    # Update original dataframe safely
    raw_data_1_col_nan_dropped_matm.update(dataframe_block)
    return raw_data_1_col_nan_dropped_matm

In [ ]:
%%time
# Applying the function to the dataframe
df7=raw_data_1_col_nan_dropped_matm.copy()
raw_data_1_col_nan_dropped_matm = PdCu_lattice(df7)

In [ ]:
# The application was successfull
%%time
raw_data_1_col_nan_dropped_matm.isnull().sum()

In [ ]:
%%time
def pymatcomp_obj_to_mpformula(x):
    """
    Converts a pymatgen Composition object (from StrToComposition)
    into a formula string with element order preserved.

    Parameters:
    - x: Composition object

    Returns:
    - str: string formula like 'Pd47.25Cu52.75'
    """
    comp_dict = x.as_dict()
    return ''.join(f"{el}{round(amt, 2)}" for el, amt in comp_dict.items())

In [ ]:
%%time
# We create a new column "Membrane Formula".
raw_data_1_col_nan_dropped_matm['Membrane Formula']=raw_data_1_col_nan_dropped_matm['Composition_matrix'].apply(pymatcomp_obj_to_mpformula)
s=raw_data_1_col_nan_dropped_matm['Membrane Formula']
raw_data_1_col_nan_dropped_matm.pop('Membrane Formula')
raw_data_1_col_nan_dropped_matm.insert(4,'Membrane Formula', s)

In [ ]:
%%time
# All the columns of Pd have been appended 'Pd1.0' which is not elegant. We replace it by 'Pd'.
raw_data_1_col_nan_dropped_matm['Membrane Formula']=raw_data_1_col_nan_dropped_matm['Membrane Formula'].where(raw_data_1_col_nan_dropped_matm['Membrane Formula']!='Pd1.0', 'Pd')

In [ ]:
%%time
# Checking
raw_data_1_col_nan_dropped_matm['Membrane Formula'].unique()

In [ ]:
# Drop duplicated rows, if there are still any
%%time
print('The shape of the dataframe before duplicated rows dropping:', raw_data_1_col_nan_dropped_matm.shape)
df8=raw_data_1_col_nan_dropped_matm.copy()
# df8[df8.iloc[:,5::].duplicated()==True]
to_show = ['Thickness (micron)', 'Lattice parameter (nm)', 'Temperature (°C)', 'Pressure difference P1^n-P2^n (Pa^n)', 'Pressure exponent (n)', 'Hydrogen permeability (mol.m^-1.s^-1.Pa^-n)']
df8=df8.drop_duplicates(subset=to_show, ignore_index=True)
raw_data_1_col_nan_dropped_matm=df8
print('The shape of the dataframe after duplicated rows dropped:', raw_data_1_col_nan_dropped_matm.shape)

In [ ]:
# Utility function to convert unit into SI
%%time
def micron_to_meter(x):
    return np.nan if pd.isnull(x) else x / (1e6)

def celsius_to_kelvin(x):
    return np.nan if pd.isnull(x) else x + 273.15

def nanometer_to_meter(x):
    return np.nan if pd.isnull(x) else x / (1e9)

In [ ]:
# Unit conversion
%%time
raw_data_1_col_nan_dropped_matm[['Thickness (micron)', 'Temperature (°C)',  'Lattice parameter (nm)']]= raw_data_1_col_nan_dropped_matm.transform({
    'Thickness (micron)':micron_to_meter,
    'Temperature (°C)':celsius_to_kelvin,
    'Lattice parameter (nm)':nanometer_to_meter
}
)
rename_dict = {'Thickness (micron)':'Thickness (m)', 'Temperature (°C)':'Temperature (K)', 'Lattice parameter (nm)':'Lattice parameter (m)', 'Pressure exponent (n)':'Pressure exponent n', 'Hydrogen permeability (mol.m^-1.s^-1.Pa^-n)':'Permeability (mol/m/s/Pa^n)'}

raw_data_1_col_nan_dropped_matm=raw_data_1_col_nan_dropped_matm.rename(columns=rename_dict)
raw_data_1_col_nan_dropped_matm.head()

In [ ]:
%%time
bool_mask = raw_data_1_col_nan_dropped_matm['Composition in mole percent']!=raw_data_1_col_nan_dropped_matm['Membrane Formula']
raw_data_1_col_nan_dropped_matm[bool_mask]

In [ ]:
%%time
# Drop the 'Composition in mole percent' column because its content is same as 'Membrane Formula', but the latter is better because created using pymatgen composition #object
df9=raw_data_1_col_nan_dropped_matm.copy()
df9=df9.drop(columns=['Composition in mole percent'])
raw_data_1_col_nan_dropped_matm=df9

In [ ]:
%%time
# Renaming the 'Membrane Formula' column into 'Composition in mole percent'.
raw_data_1_col_nan_dropped_matm=raw_data_1_col_nan_dropped_matm.rename(columns={'Membrane Formula':'Composition in mole percent'})

In [ ]:
%%time
# Check few raw if everything worked perfectly
raw_data_1_col_nan_dropped_matm.head()

In [ ]:
raw_data_1_col_nan_dropped_matm['Composition in mole percent'].shape

In [ ]:
# len(raw_data_1_col_nan_dropped_matm['Reference'].unique())

In [ ]:
# Define your directory (I save using pickle because it conserved my datatype from notebooks to notebooks, while csv can't)
%%time

# Define filename
filename = 'Pd_membrane_data_preprocessed.pkl'

# Define save directory
out_path = '/content/drive/MyDrive/PhD_Pd alloy prediction/Resut_Data_cleaning'

# Build the full path safely
file_path = os.path.join(out_path, filename)

# Save the DataFrame
raw_data_1_col_nan_dropped_matm.to_pickle(file_path)

# Load it later
# df_loaded = pd.read_pickle(file_path)

In [ ]:
# # Saving the dataset
# %%time
# # Define output path
# out_path = '/content/drive/MyDrive/PhD_Pd alloy prediction/Resut_Data_cleaning/Pd_Membrane_Preprocessed.csv'

# # Save DataFrame to CSV
# raw_data_1_col_nan_dropped_matm.to_csv(out_path, index=False)

# # Optional: Confirm it was saved
# print(f"Data successfully saved to: {out_path}")